In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import librosa.display

PATH_0 = "../birdclef-2022-data"
PATH_1 = "birdclef-2022-data"

meta = pd.read_csv(f"{PATH_1}/train_metadata.csv")

base_path = f'{PATH_1}/train_audio/'
audio_name = f'{PATH_1}/train_audio/afrsil1/XC125458.ogg'
image_directory = f'{PATH_1}/train_images/'
hop_length = 512
window_size = 1024

meta.head()

In [ ]:
y, sr = librosa.load(audio_name)
window = np.hanning(window_size)
out  = librosa.core.spectrum.stft(y, n_fft = window_size, hop_length = hop_length,
       window=window)
out = 2 * np.abs(out) / np.sum(window)

librosa.display.specshow(librosa.amplitude_to_db(out,ref=np.max),
               y_axis='log', x_axis='time')

In [ ]:
import skimage.io

def spectrogram_image(y, sr, out, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
                                            n_fft=hop_length*2, hop_length=hop_length)
    mels = np.log(mels + 1e-9) # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy

    # save as PNG
    skimage.io.imsave(image_directory + out + '.png', img)

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def load_batch_spectograms(audio_names):
    for entry in audio_names:
        y, sr = librosa.load(base_path + entry)
        spectrogram_image(y, sr, entry[:-4], hop_length, 256)

In [ ]:
spec = spectrogram_image(y, sr, hop_length, 256)
plt.imshow(spec)

In [ ]:
# careful, this takes multiple hours to run

def generate_spectograms():

    names = meta['filename']
    load_batch_spectograms(names)


In [ ]:
import os
def add_folder_structure():
    names = meta['primary_label'].unique()
    for name in names:
        os.mkdir(image_directory + name)

In [ ]:
add_folder_structure()